# Standardize output data
Run these cells sequentially

In [ ]:
# Import necessary modules
import os
from utils import *
import utils.for_setpath as path
import utils.for_empatica as empatica

# Data processing
import numpy as np
import matplotlib.pyplot as plt

# Pluma-python API  
from modules import *
from pluma.schema.outdoor import build_schema
from missing_sync import build_schema

# Export lsl, geodata, and empatica streams to CSV

Geodata is the backbone of this process since it is already sampled at 1 Hz and without it, it's impossible to synchronize all the data streams 

In [ ]:
import os
from utils import *
import utils.for_setpath as path
import utils.for_empatica as empatica
import utils.for_climate as climate

# ---------------------------------------------------------------------
# MAIN SCRIPT
datadir    = os.path.join(path.sourcedata, 'data')
for participant_folder in os.listdir(datadir):
    if participant_folder.startswith("OE"):
        participant_path = os.path.join(datadir, participant_folder)
        for session_folder in os.listdir(participant_path):
            session_path = os.path.join(participant_path, session_folder)
            if os.path.isdir(session_path):
                session_name = extract_session_name(session_folder)
                # Geodata output
                output = os.path.join(path.sourcedata, 'supp', 'geodata', 'log', f'sub-{participant_folder}', f'ses-{session_name}')
                if not os.path.exists(output): 
                    os.makedirs(output)
                # Empatica output
                csv_outdir = os.path.join(path.sourcedata, 'supp', 'stress_csv', f'sub-{participant_folder}', f'ses-{session_name}')
                if not os.path.exists(csv_outdir): 
                    os.makedirs(csv_outdir)
                try: # Try to load the dataset
                    datapicker = create_datapicker(path = session_path, schema=build_schema)
                    dataset    = load_dataset(datapicker.selected_path,schema=build_schema)
                except:
                    try: # Try to load by other schema
                        print("Trying to load by other schema")
                        datapicker = create_datapicker(path=session_path, schema=build_schema, calibrate_ubx_to_harp=False)
                        dataset    = load_dataset(datapicker.selected_path, ubx=True, unity=False, calibrate_ubx_to_harp=False, schema=build_schema)
                    except:
                        print(f"Error in {participant_folder}-{session_name}")
                        continue
                # Save geodata to csv
                climate.geodata_to_csv(dataset, participant_folder, session_name, output)
                # Save empatica data to csv
                empatica.empatica_and_ecg_to_csv(dataset, csv_outdir) 
                print(f'Successfully saved data for {participant_folder}-{session_name}!')

# Export processed empatica at 1 Hz

In [ ]:
# ---------------------------------------------------------------------
# MAIN SCRIPT
datadir = os.path.join(path.sourcedata, 'data')
for participant_folder in os.listdir(datadir):
    if participant_folder.startswith("OE"):
        participant_path = os.path.join(datadir, participant_folder)
        for session_folder in os.listdir(participant_path):
            session_path = os.path.join(participant_path, session_folder)
            if os.path.isdir(session_path):
                session_name = extract_session_name(session_folder)
                input_directory = os.path.join(path.sourcedata, 'supp', 'stress_csv',f'sub-{participant_folder}', f'ses-{session_name}')
                output_directory = os.path.join(input_directory, '_1hz')
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)
                # Resample Empatica data
                try:
                    empatica.export_resampled_empatica_data(input_directory, output_directory)
                except:
                    print(f"Error in {participant_folder}-{session_name}")
                    continue

# Export Eye-tracking metrics (to do)

In [ ]:
import os
from utils import *
import utils.for_setpath as path
import utils.for_eye_tracker as et

# ---------------------------------------------------------------------
# MAIN SCRIPT
datadir = os.path.join(path.sourcedata, 'data')
for participant_folder in os.listdir(datadir):
    if participant_folder.startswith("OE"):
        participant_path = os.path.join(datadir, participant_folder)
        for session_folder in os.listdir(participant_path):
            session_path = os.path.join(participant_path, session_folder)
            if os.path.isdir(session_path):
                session_name = extract_session_name(session_folder)
                csv_outdir = os.path.join(path.sourcedata, 'supp', 'gaze', f'sub-{participant_folder}', f'ses-{session_name}')
                if not os.path.exists(csv_outdir): 
                    os.makedirs(csv_outdir)
                try: # Try to load the dataset
                    datapicker = create_datapicker(path = session_path, schema=build_schema)
                    dataset    = load_dataset(datapicker.selected_path,schema=build_schema)
                except:
                    try: # Try to load by other schema
                        print("Trying to load by other schema")
                        datapicker = create_datapicker(path=session_path, schema=build_schema, calibrate_ubx_to_harp=False)
                        dataset    = load_dataset(datapicker.selected_path, ubx=True, unity=False, calibrate_ubx_to_harp=False, schema=build_schema)
                    except:
                        print(f"Error in {participant_folder}-{session_name}")
                        continue
                try:
                    # Save the data to csv
                    et.export_gaze_to_csv(dataset, csv_outdir) 
                    print(f'Successfully saved data for {participant_folder}-{session_name}!')
                except:
                    print(f"Error in exporting gaze data for {participant_folder}-{session_name}")
                    continue

# Add EEG and other informations to the table data
Band power metrics have been computed on preprocessed EEG data using MATLAB. Other data like participant_id, age, sex, and path name are also to be added.
Implemented:
- EEG
- Geodata
- Empatica

To do:
- Add eye-tracking
- Add HR
- Add behavioral and phenotype data

In [ ]:
import os
import pandas as pd
from utils import *
import utils.for_setpath as path

def tidy_data(data, subject_id, session_id, bidsroot):
    """Tidy the tabular data
    Add participant id
    Add session id
    Add age
    Add sex
    Add path typology
    Make gps columns (lon,lat,alt) the 2nd, 3rd, and 4th columns
    Rename columns to be more informative
    """
    # Add metadata
    data['participant_id'] = subject_id
    data['session_id'] = session_id
    participants = pd.read_csv(os.path.join(bidsroot, 'participants.tsv'), sep='\t')
    data['age'] = participants[participants['participant_id'] == f'sub-{subject_id}']['age'].values[0]
    data['sex'] = participants[participants['participant_id'] == f'sub-{subject_id}']['sex'].values[0]

    # Add behavioral ratings
    subj_num            = int(subject_id[2:])
    id                  = infer_participant_code('lisbon', subj_num, session_id, 'all')
    sam                 = fetch_beh_ratings(os.path.join(path.sourcedata,'supp'), id)
    if sam.size == 0:  # If no behavioral data, add empty columns
        data['valence']     = np.nan
        data['arousal']     = np.nan
        data['naturalness'] = np.nan
        data['crowdedness'] = np.nan
    else:  # Otherwise, add behavioral ratings
        sam                 = np.tile(sam, (len(data), 1))
        data['valence']     = sam[:, 0]
        data['arousal']     = sam[:, 1]
        data['naturalness'] = sam[:, 2]
        data['crowdedness'] = sam[:, 3]

    # Rename columns
    rename_dict = {
        # Time
        'DateTime': 'time',

        # Climate
        'tk_airquality_iaqindex_value': 'air_quality_index',
        'tk_airquality_temperature_value': 'air_temperature',
        'tk_airquality_humidity_value': 'air_humidity',
        'tk_airquality_airpressure_value': 'air_pressure',
        'tk_soundpressurelevel_spl_value': 'sound_pressure_level',
        'tk_humidity_humidity_value': 'humidity_sensor',
        'tk_analogin_voltage_value': 'analog_voltage',
        'tk_particulatematter_pm1_0_value': 'pm1_0',
        'tk_particulatematter_pm2_5_value': 'pm2_5',
        'tk_particulatematter_pm10_0_value': 'pm10_0',
        'tk_dual0_20ma_solarlight_value': 'solar_light',
        'tk_thermocouple_temperature_value': 'thermocouple_temp',
        'tk_ptc_airtemp_value': 'ptc_air_temp',
        'atmos_northwind_value': 'north_wind',
        'atmos_eastwind_value': 'east_wind',
        'atmos_gustwind_value': 'gust_wind',
        'atmos_airtemperature_value': 'atmospheric_temperature',
        'temp_atmos': 'temp_atmospheric',
        'temp_tk': 'temp_tk',
        'temp_tk_ptc': 'temp_tk_ptc',
        'temp_radiant': 'radiant_temp',
        'hPa': 'pressure',
        'dew_point': 'dew_point_temperature',

        # Empatica
        'E4_HR_IBI': 'empatica_heart_rate',
        'TEMP': 'skin_temperature',
        'EDA_RAW': 'eda_raw',
        'EDA_PHASIC': 'eda_phasic',
        'AccX': 'x_acceleration',
        'AccY': 'y_acceleration',
        'AccZ': 'z_acceleration',
        'Magnitude': 'acceleration_magnitude',
        'BVP_Values': 'bvp',

        # Other
        'stress_category': 'utci_stress_category'
    }
    data.rename(columns=rename_dict, inplace=True)

    # Dynamically reorder columns
    column_order = [
        'time',  # Time-related column
        'longitude', 'latitude', 'elevation',  # GPS columns
        'participant_id', 'session_id', 'age', 'sex'  # Metadata columns
    ]
    # Identify remaining columns dynamically
    other_columns = [col for col in data.columns if col not in column_order]
    reordered_columns = column_order + other_columns

    # Apply the reordered column structure
    data = data[reordered_columns]

    return data


# ---------------------------------------------------------------------
# MAIN SCRIPT
datadir = os.path.join(path.sourcedata, 'data')
for participant_folder in os.listdir(datadir):
    if participant_folder.startswith("OE"):
        participant_path = os.path.join(datadir, participant_folder)
        for session_folder in os.listdir(participant_path):
            session_path = os.path.join(participant_path, session_folder)
            if os.path.isdir(session_path):
                session_name = extract_session_name(session_folder)
                # Define paths of streams
                empatica_data_path = os.path.join(path.sourcedata, 'supp', 'stress_csv', f'sub-{participant_folder}', f'ses-{session_name}', '_1hz', 'data_all_1Hz.csv') 
                geodata_data_path  = os.path.join(path.sourcedata, 'supp', 'geodata', 'log', f'sub-{participant_folder}', f'ses-{session_name}', f'sub-{participant_folder}_ses-{session_name}_geodata.xlsx')
                eeg_data_path      = os.path.join(path.results, 'analysis-climate_physio_eeg_model_pipeline-outdoor_literature', f'sub-{participant_folder}', f'ses-{session_name}', 'data', 'a01_psd_compute.m', 'eeg_power.xlsx')
                # Define output path
                output_path        = os.path.join(path.results, 'fulldata', f'sub-{participant_folder}', f'ses-{session_name}')
                if not os.path.exists(output_path): 
                    os.makedirs(output_path)
                # Attempt to merge the data (first empatica and geodata, and then eeg)
                # If any file is missing skip to the next data stream
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                #                             GEODATA                           #
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                # Load Geodata
                print("Loading Geodata...")
                try:
                    geodata_data = pd.read_excel(geodata_data_path,parse_dates=['time'])
                except:
                    print("There is no Geodata for this session")
                    continue
                geodata_data = geodata_data.rename(columns={'time':'DateTime'})
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                #                            EMPATICA                           #
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                # Load Empatica data
                print("Loading Empatica data...")
                try:
                    empatica_data = pd.read_csv(empatica_data_path,parse_dates=['DateTime'])
                except:   
                    print("There is no Empatica data for this session")
                    print("Adding empty Empatica data...")
                    # Create an empty DataFrame with the same column structure
                    empatica_columns = [
                        "DateTime", "E4_HR", "E4_HR_IBI", "TEMP", "EDA_RAW",
                        "EDA_PHASIC", "AccX", "AccY", "AccZ", "Magnitude", "BVP_Values"
                    ]
                    empatica_data = pd.DataFrame(columns=empatica_columns)
                    empatica_data['DateTime'] = geodata_data['DateTime']
                # Merge Empatica data with climate data on 'DateTime'
                print("Merging Empatica data with climate data...")
                combined_data = pd.merge(empatica_data, geodata_data, on='DateTime', how='inner')
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                #                            EEG                                #
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                # Load EEG data
                print("Loading EEG data...")
                try:
                    eeg_data = pd.read_excel(eeg_data_path)
                except:
                    print("There is no EEG data for this session")
                    print("Adding empty EEG data...")
                    # Create an empty DataFrame with the same column structure
                    eeg_columns = [
                        "Time", "delta", "theta", "alpha", "beta", "gamma",
                        "frontal alpha", "frontal midline theta", "theta-beta ratio",	
                        "frontal alpha asymmetry", "frontal theta"
                    ]
                    eeg_data = pd.DataFrame(columns=eeg_columns)
                eeg_data = eeg_data.rename(columns={'Time':'DateTime'})
                if eeg_data.empty:
                    eeg_data['DateTime'] = combined_data['DateTime']
                    # Merge EEG data with combined data on 'DateTime'
                    print("Merging empty EEG data with combined data...")
                    combined_data = pd.merge(combined_data, eeg_data, on='DateTime', how='left')
                else:
                    # Merge EEG data with combined data on 'DateTime'
                    print("Merging EEG data with combined data...")
                    combined_data = pd.merge(combined_data, eeg_data, on='DateTime', how='inner')
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                #                            TIDY                               #
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
                # Tidy combined data
                print("Tidying combined data...")
                combined_data = tidy_data(combined_data, participant_folder, session_name, path.bidsroot)
                # Export combined data
                print("Exporting combined data...")
                combined_data.to_csv(os.path.join(output_path, 'alldata.csv'), index=False)
                


# TESTS BELOW

##### Correlations (Physiological and UTCI)

In [ ]:
# Import necessary modules
import pandas as pd
import numpy as np
import os
from scipy.stats import pearsonr, spearmanr, linregress
import matplotlib.pyplot as plt

# Set plotting parameters
plt.rcParams.update({'font.size': 12})

# Define paths and session information
session_name = 'Lapa' # Replace with your session name
subject_folder = 'sub-OE009' # Replace with your subject folder name
# Update these variables according to your directory structure
sourcedata_path = os.path.join(path.sourcedata, 'supp') # Replace with the path to 'sourcedata' directory
input_directory = os.path.join(path.sourcedata, 'supp', 'stress_csv', subject_folder, 'ses-'+session_name) 
empatica_data_path = os.path.join(input_directory, '_1hz', 'data_all_1Hz.csv')
geodata_data_path = os.path.join(sourcedata_path, 'geodata', 'log', subject_folder, 'ses-'+session_name, f'sub-{participant_folder}', f'ses-{session_name}')
output_dir = os.path.join(path.sourcedata, 'supp', 'correlation', subject_folder, 'ses-'+session_name)       

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load Empatica data
print("Loading Empatica data...")
empatica_data = pd.read_csv(empatica_data_path, parse_dates=['DateTime'])

# Ensure 'DateTime' is in datetime format
if empatica_data['DateTime'].dtype != 'datetime64[ns]':
    empatica_data['DateTime'] = pd.to_datetime(empatica_data['DateTime'], errors='coerce')

# Remove any rows with NaT in 'DateTime'
empatica_data = empatica_data.dropna(subset=['DateTime'])

# Floor to seconds to ensure alignment
empatica_data['DateTime'] = empatica_data['DateTime'].dt.floor('S')

# --- Climate Data Processing ---

# Locate the climate data CSV file
print("Locating climate data CSV file...")
log2_path = os.path.join(sourcedata_path, 'log2')
subject_path = os.path.join(log2_path, subject_folder)

# Find session folder containing the session name
session_folder = None
for folder in os.listdir(subject_path):
    if session_name in folder:
        session_folder = folder
        break

if session_folder is None:
    print(f"Session folder containing '{session_name}' not found in {subject_path}.")
else:
    session_path = os.path.join(subject_path, session_folder)

    # Find the CSV file containing 'geodata_processed' in the filename
    climate_csv_file = None
    for file in os.listdir(session_path):
        if 'geodata_processed' in file and file.endswith('.csv'):
            climate_csv_file = file
            break

    if climate_csv_file is None:
        print(f"No CSV file containing 'geodata_processed' found in {session_path}.")
    else:
        climate_csv_path = os.path.join(session_path, climate_csv_file)
        print(f"Climate data CSV file found: {climate_csv_path}")

        # Read the climate data CSV file
        print("Reading climate data...")
        climate_data = pd.read_csv(climate_csv_path)

        # Ensure datetime column is parsed correctly
        if 'DateTime' in climate_data.columns:
            climate_data['DateTime'] = pd.to_datetime(climate_data['DateTime'], errors='coerce')
        else:
            print("No 'DateTime' column found in climate data. Please ensure the CSV contains datetime information.")
            climate_data = None

        # Remove any rows with NaT in 'DateTime'
        if climate_data is not None:
            climate_data = climate_data.dropna(subset=['DateTime'])

        # Check if 'utci' column exists
        if climate_data is not None and 'utci' not in climate_data.columns:
            print("'utci' column not found in climate data.")
            climate_data = None

        if climate_data is not None:
            # Check if 'second' column exists
            if 'second' in climate_data.columns:
                # Combine 'DateTime' and 'second' to create a full datetime with seconds
                print("Combining 'DateTime' and 'second' to create full datetime with seconds...")
                # Floor 'DateTime' to the nearest minute
                climate_data['DateTime'] = climate_data['DateTime'].dt.floor('min')
                # Add 'second' column as timedelta
                climate_data['DateTime'] += pd.to_timedelta(climate_data['second'], unit='s')
                # Now 'DateTime' includes seconds
            else:
                print("'second' column not found in climate data.")
                climate_data = None

            # Ensure 'DateTime' is in datetime format
            if climate_data is not None and climate_data['DateTime'].dtype != 'datetime64[ns]':
                climate_data['DateTime'] = pd.to_datetime(climate_data['DateTime'], errors='coerce')
                climate_data = climate_data.dropna(subset=['DateTime'])

            # Check data type
            print("Climate data 'DateTime' dtype:", climate_data['DateTime'].dtype)

            # Proceed if 'DateTime' and 'utci' are available
            if climate_data is not None:
                # Merge Empatica data with climate data on 'DateTime'
                print("Merging Empatica data with climate data...")
                combined_data = pd.merge(empatica_data, climate_data[['DateTime', 'utci']], on='DateTime', how='inner')

                # Save the combined data to CSV with proper datetime formatting
                combined_data.to_csv(os.path.join(output_dir, 'combined_data.csv'), index=False, date_format='%Y-%m-%d %H:%M:%S')

                # Perform correlations between 'utci' and Empatica variables
                print("Performing correlations between 'utci' and Empatica variables...")
                empatica_vars = combined_data.select_dtypes(include=[np.number]).columns.drop('utci')
                correlation_results = []

                for var in empatica_vars:
                    # Drop NaN values for the pair
                    valid_data = combined_data[['utci', var]].dropna()
                    if len(valid_data) < 2:
                        print(f"Not enough data to compute correlation between 'utci' and '{var}'.")
                        continue

                    # Pearson correlation
                    pearson_corr, pearson_p = pearsonr(valid_data['utci'], valid_data[var])

                    # Spearman correlation
                    spearman_corr, spearman_p = spearmanr(valid_data['utci'], valid_data[var])

                    # Append results
                    correlation_results.append({
                        'Variable': var,
                        'Pearson_Correlation': pearson_corr,
                        'Pearson_p_value': pearson_p,
                        'Spearman_Correlation': spearman_corr,
                        'Spearman_p_value': spearman_p
                    })

                    # Plot scatter plot with best-fit line
                    plt.figure(figsize=(8, 6))

                    # Scatter plot
                    plt.scatter(valid_data['utci'], valid_data[var], color='blue', alpha=0.6, edgecolor='k', label='Data')

                    # Best-fit line using linear regression (OLS)
                    slope, intercept, r_value, p_value, std_err = linregress(valid_data['utci'], valid_data[var])
                    x_vals = np.array([valid_data['utci'].min(), valid_data['utci'].max()])
                    y_vals = intercept + slope * x_vals
                    plt.plot(x_vals, y_vals, color='red', linewidth=2, label='OLS Linear Regression')

                    # Aesthetics adjustments
                    plt.xlabel('Universal Thermal Climate Index (UTCI)', fontsize=14)
                    plt.ylabel(var, fontsize=14)
                    plt.title(f'Relationship between {var} and UTCI', fontsize=16)
                    plt.legend(frameon=False, fontsize=12)

                    # Remove background grid and frame
                    plt.gca().spines['top'].set_visible(False)
                    plt.gca().spines['right'].set_visible(False)
                    plt.gca().spines['left'].set_linewidth(1.5)
                    plt.gca().spines['bottom'].set_linewidth(1.5)

                    # Include Pearson and Spearman correlation coefficients and p-values in the plot
                    textstr = '\n'.join((
                        r'Pearson $r=%.2f$ (p=%.2e)' % (pearson_corr, pearson_p),
                        r'Spearman $\rho=%.2f$ (p=%.2e)' % (spearman_corr, spearman_p)
                    ))
                    # Place text box in upper left in axes coords
                    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=12,
                            verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

                    # Tight layout for better spacing
                    plt.tight_layout()

                    # Save the figure
                    plt.savefig(os.path.join(output_dir, f'scatter_{var}_vs_utci.png'), dpi=300)
                    plt.close()

                # Save correlation results to CSV
                print("Saving correlation results...")
                correlation_df = pd.DataFrame(correlation_results)
                correlation_df.to_csv(os.path.join(output_dir, 'correlation_results.csv'), index=False)

                print("Correlation analysis complete. Results saved to the output directory.")
            else:
                print("Climate data could not be processed due to missing 'DateTime', 'utci', or 'second' column.")


##### Covariance Matrices
Perform correlations for:
- within subjects
- within paths
- between paths
- between subjects

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import pearsonr, spearmanr, linregress
import matplotlib.pyplot as plt

def compute_covariance_matrix(data, method='pairwise'):
    """
    Compute covariance matrix using different methods to handle missing data.

    Args:
        data (pd.DataFrame): DataFrame with numeric variables.
        method (str): Method to handle missing values. Options are:
            - 'pairwise': Computes pairwise covariances, ignoring missing values.
            - 'mean_impute': Fills missing values with the column mean.
            - 'median_impute': Fills missing values with the column median.
            - 'interpolate': Uses linear interpolation to fill missing values.

    Returns:
        pd.DataFrame: Covariance matrix.
    """
    if method == 'pairwise':
        # Pandas computes pairwise covariances by default
        return data.cov()
    elif method == 'mean_impute':
        filled_data = data.fillna(data.mean())
    elif method == 'median_impute':
        filled_data = data.fillna(data.median())
    elif method == 'interpolate':
        filled_data = data.interpolate(method='linear', limit_direction='both')
    else:
        raise ValueError(f"Unknown method: {method}")
    return filled_data.cov()

def process_data(empatica_data_path, climate_data_path, output_dir):
    
    # Load Empatica data
    empatica_data = pd.read_csv(empatica_data_path, parse_dates=['DateTime'])
    empatica_data['DateTime'] = pd.to_datetime(empatica_data['DateTime'], errors='coerce')
    empatica_data = empatica_data.dropna(subset=['DateTime']).set_index('DateTime')

    # Load Climate data
    climate_data = pd.read_excel(climate_data_path, parse_dates=['time'])
    climate_data['time'] = pd.to_datetime(climate_data['time'], errors='coerce')
    climate_data = climate_data.dropna(subset=['time']).set_index('time')

    # Merge data
    combined_data = pd.merge(empatica_data, climate_data[['utci']], left_index=True, right_index=True, how='inner')
    combined_data.to_csv(os.path.join(output_dir, 'combined_data.csv'))

    # Select numeric columns for covariance analysis
    numeric_data = combined_data.select_dtypes(include=[np.number])

    # Compute covariance matrices with different methods
    covariance_methods = ['pairwise', 'mean_impute', 'median_impute', 'interpolate']
    for method in covariance_methods:
        print(f"Computing covariance matrix using {method} method...")
        covariance_matrix = compute_covariance_matrix(numeric_data, method=method)
        covariance_matrix.to_csv(os.path.join(output_dir, f'covariance_matrix_{method}.csv'))
        print(f"Covariance matrix ({method}) saved.")

    print("Covariance matrix computations complete.")

def main():
    # Define paths
    session_name = 'Lapa'
    subject_folder = 'sub-OE009'
    output_dir = os.path.join(path.sourcedata, 'supp', 'correlation', subject_folder, f'ses-{session_name}')
    empatica_data_path = os.path.join(path.sourcedata, 'supp', 'stress_csv', subject_folder, f'ses-{session_name}', '_1hz', 'data_all_1Hz.csv')
    climate_data_path = os.path.join(path.sourcedata, 'supp', 'geodata','log', subject_folder, f'ses-{session_name}', f'{subject_folder}_ses-{session_name}_geodata.xlsx')

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process data and compute covariance matrices
    process_data(empatica_data_path, climate_data_path, output_dir)

if __name__ == "__main__":
    main()


In [ ]:
from utils import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, ttest_rel, pearsonr, spearmanr

###############################################################################
# Choose which columns from geodata to include (besides "time") 
# Typically, you want numeric columns that might be correlated with physiological data.
# Exclude obviously irrelevant columns like 'geometry' or duplicates.
###############################################################################
SELECTED_GEODATA_COLS = [
    # Potential environment/climate columns
    "tk_airquality_iaqindex_value","tk_airquality_temperature_value","tk_airquality_humidity_value",
    "tk_airquality_airpressure_value","tk_soundpressurelevel_spl_value","tk_humidity_humidity_value",
    "tk_analogin_voltage_value","tk_particulatematter_pm1_0_value","tk_particulatematter_pm2_5_value",
    "tk_particulatematter_pm10_0_value","tk_dual0_20ma_solarlight_value","tk_thermocouple_temperature_value",
    "tk_ptc_airtemp_value","atmos_northwind_value","atmos_eastwind_value","atmos_gustwind_value",
    "atmos_airtemperature_value","accelerometer_orientation_x","accelerometer_orientation_y","accelerometer_orientation_z",
    "accelerometer_gyroscope_x","accelerometer_gyroscope_y","accelerometer_gyroscope_z","accelerometer_linearaccl_x",
    "accelerometer_linearaccl_y","accelerometer_linearaccl_z","accelerometer_magnetometer_x","accelerometer_magnetometer_y",
    "accelerometer_magnetometer_z","accelerometer_accl_x","accelerometer_accl_y","accelerometer_accl_z",
    "accelerometer_gravity_x","accelerometer_gravity_y","accelerometer_gravity_z",
    # Merged columns from subsequent steps
    "empatica_e4_gsr","empatica_e4_hr","empatica_e4_ibi","empatica_e4_temperature",
    "humidity","wind_speed","temp_atmos","temp_tk","temp_tk_ptc","temp_radiant","utci",
    "day_of_year","ghi","hPa","dew_point","solar_altitude","solar_azimuth","mrt"
]

###############################################################################
def compute_covariance_matrix(data, method='pairwise'):
    """
    Compute covariance matrix using different methods to handle missing data.
    """
    if method == 'pairwise':
        # Pandas defaults to pairwise complete for .cov()
        return data.cov()
    elif method == 'mean_impute':
        filled_data = data.fillna(data.mean(numeric_only=True))
    elif method == 'median_impute':
        filled_data = data.fillna(data.median(numeric_only=True))
    elif method == 'interpolate':
        filled_data = data.interpolate(method='linear', limit_direction='both')
    else:
        raise ValueError(f"Unknown method: {method}")
    return filled_data.cov()

def compute_correlation_matrix(data, method='pearson'):
    """
    Compute correlation matrix: 'pearson', 'spearman', or 'kendall'.
    """
    return data.corr(method=method)

def plot_heatmap(matrix, title, output_path):
    """
    Plots a heatmap of the given matrix (DataFrame).
    """
    plt.figure(figsize=(12,10))
    sns.heatmap(matrix, annot=False, cmap='coolwarm', square=True)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(output_path, dpi=150)
    plt.close()

def process_data(empatica_data_path, climate_data_path, output_dir):
    
    ###############################################################################
    # Load Empatica Data
    ###############################################################################
    empatica_data = pd.read_csv(empatica_data_path, parse_dates=['DateTime'])
    empatica_data['DateTime'] = pd.to_datetime(empatica_data['DateTime'], errors='coerce')
    empatica_data.dropna(subset=['DateTime'], inplace=True)
    empatica_data.set_index('DateTime', inplace=True)

    ###############################################################################
    # Load "geodata" / climate data
    ###############################################################################
    climate_data = pd.read_excel(climate_data_path, parse_dates=['time'])
    climate_data['time'] = pd.to_datetime(climate_data['time'], errors='coerce')
    climate_data.dropna(subset=['time'], inplace=True)
    climate_data.set_index('time', inplace=True)

    ###############################################################################
    # Merge
    # We'll gather columns from climate_data that we want 
    # (some may not exist, so we will safely drop missing)
    ###############################################################################
    available_cols = [c for c in SELECTED_GEODATA_COLS if c in climate_data.columns]
    # If none found, we proceed with a minimal set
    if not available_cols:
        print("No selected columns found in climate_data. Falling back to just utci if present.")
        if "utci" in climate_data.columns:
            available_cols = ["utci"]
        else:
            print("No utci in climate data; the merge will produce minimal columns.")
            available_cols = climate_data.columns  # fallback to all

    # Subset climate_data
    climate_subset = climate_data[available_cols]

    # Merge
    combined_data = pd.merge(empatica_data, climate_subset, 
                             left_index=True, right_index=True, how='inner')

    # Export the final table
    combined_data.to_csv(os.path.join(output_dir, 'combined_data_full.csv'))
    print(f"Exported final combined data with {len(combined_data)} rows, {combined_data.shape[1]} columns.")

    ###############################################################################
    # Covariance / Correlation
    ###############################################################################
    # We'll select numeric columns only
    numeric_data = combined_data.select_dtypes(include=[np.number]).copy()

    # Possibly we drop columns that have zero variance or near-constant
    # or too many NaNs, but let's keep it simple

    # 1) Covariance with different methods
    covariance_methods = ['pairwise', 'mean_impute', 'median_impute', 'interpolate']
    for method in covariance_methods:
        cov_mat = compute_covariance_matrix(numeric_data, method=method)
        cov_mat.to_csv(os.path.join(output_dir, f'cov_matrix_{method}.csv'))
        plot_heatmap(cov_mat, f"Covariance Matrix [{method}]", 
                     os.path.join(output_dir, f'cov_matrix_{method}.png'))

    # 2) Correlation (Pearson, Spearman)
    for corr_method in ['pearson','spearman']:
        corr_mat = compute_correlation_matrix(numeric_data, method=corr_method)
        corr_mat.to_csv(os.path.join(output_dir, f'corr_matrix_{corr_method}.csv'))
        plot_heatmap(corr_mat, f"Correlation Matrix [{corr_method}]", 
                     os.path.join(output_dir, f'corr_matrix_{corr_method}.png'))

    print("Covariance and correlation matrices saved & plotted.")

def main():

    # ---------------------------------------------------------------------
    # MAIN SCRIPT
    datadir = os.path.join(path.sourcedata, 'data')
    for participant_folder in os.listdir(datadir):
        if participant_folder.startswith("OE"):
            participant_path = os.path.join(datadir, participant_folder)
            for session_folder in os.listdir(participant_path):
                session_path = os.path.join(participant_path, session_folder)
                if os.path.isdir(session_path):
                    session_name = extract_session_name(session_folder)
                    subject_folder = f'sub-{participant_folder}'
                    output_dir = os.path.join(path.sourcedata, 'supp', 'correlation', subject_folder, f'ses-{session_name}')
                    empatica_data_path = os.path.join(path.sourcedata, 'supp', 'stress_csv', subject_folder, f'ses-{session_name}', '_1hz', 'data_all_1Hz.csv')
                    climate_data_path = os.path.join(path.sourcedata, 'supp', 'geodata','log', subject_folder, f'ses-{session_name}', f'{subject_folder}_ses-{session_name}_geodata.xlsx')
                    if not os.path.exists(output_dir):    
                        os.makedirs(output_dir, exist_ok=True)  
                    try:
                        print(f"Processing {participant_folder}-{session_name}...")
                        process_data(empatica_data_path, climate_data_path, output_dir)
                    except:
                        print(f"Error in {participant_folder}-{session_name}")
                        continue
                    
if __name__ == "__main__":
    main()


Session analysis

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from utils import *

###############################################################################
# Lists for separation of variables
# You can adapt these to your actual column names
###############################################################################
PHYS_VARS = [
    "empatica_e4_gsr","empatica_e4_hr","empatica_e4_ibi","empatica_e4_temperature",
    # add other physiol columns if needed
]

CLIM_VARS = [
    "tk_airquality_iaqindex_value","tk_airquality_temperature_value","tk_airquality_humidity_value",
    "tk_airquality_airpressure_value","tk_soundpressurelevel_spl_value","tk_humidity_humidity_value",
    "tk_analogin_voltage_value","tk_particulatematter_pm1_0_value","tk_particulatematter_pm2_5_value",
    "tk_particulatematter_pm10_0_value","tk_dual0_20ma_solarlight_value","tk_thermocouple_temperature_value",
    "tk_ptc_airtemp_value","atmos_northwind_value","atmos_eastwind_value","atmos_gustwind_value",
    "atmos_airtemperature_value","humidity","wind_speed","temp_atmos","temp_tk","temp_tk_ptc",
    "temp_radiant","utci","day_of_year","ghi","hPa","dew_point","solar_altitude","solar_azimuth",
    "mrt"
]

def do_within_session_correlations(combined_data, session_name, output_dir):
    """
    For a single session, compute correlation between all PHYS_VARS x CLIM_VARS,
    store in a DataFrame, and plot a heatmap.
    """
    # Keep only columns that exist
    available_phys = [v for v in PHYS_VARS if v in combined_data.columns]
    available_clim = [v for v in CLIM_VARS if v in combined_data.columns]

    if not (available_phys and available_clim):
        print(f"No overlapping columns for session={session_name}. Skipping.")
        return

    # We can do pairwise correlation row by row
    # First drop rows with all NaN in those columns
    sub_df = combined_data[available_phys + available_clim].dropna(how='all')

    # We'll create a result matrix shaped (len(phys), len(clim)) with correlation
    corr_values = np.zeros((len(available_phys), len(available_clim)))
    p_values    = np.zeros((len(available_phys), len(available_clim)))

    for i, pv in enumerate(available_phys):
        for j, cv in enumerate(available_clim):
            # Drop row if either is NaN
            pair_df = sub_df[[pv, cv]].dropna()
            if len(pair_df) > 2:
                r, p = pearsonr(pair_df[pv], pair_df[cv])
            else:
                r, p = np.nan, np.nan
            corr_values[i,j] = r
            p_values[i,j]    = p

    # Create a DataFrame
    corr_df = pd.DataFrame(corr_values, index=available_phys, columns=available_clim)
    pval_df = pd.DataFrame(p_values,  index=available_phys, columns=available_clim)

    # Save CSV
    corr_df.to_csv(os.path.join(output_dir, f"{session_name}_phys_clim_correlations.csv"))
    pval_df.to_csv(os.path.join(output_dir, f"{session_name}_phys_clim_pvalues.csv"))

    # Plot heatmap of correlation (r-values)
    plt.figure(figsize=(12,6))
    sns.heatmap(corr_df, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title(f"PHYS-CLIM Correlations (Pearson R)\nSession: {session_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{session_name}_phys_clim_correlation_heatmap.png"), dpi=150)
    plt.close()

    print(f"Session={session_name}: correlation done. Saved to {output_dir}")

def do_between_sessions_correlations(all_data, output_dir):
    """
    Compute correlation for all sessions combined (pooled data).
    We'll do the same approach as do_within_session_correlations,
    but on the entire dataset from all sessions.
    """
    available_phys = [v for v in PHYS_VARS if v in all_data.columns]
    available_clim = [v for v in CLIM_VARS if v in all_data.columns]

    if not (available_phys and available_clim):
        print("No overlapping columns for the entire dataset. Skipping between-sessions correlation.")
        return

    # Drop rows with no data in those columns
    sub_df = all_data[available_phys + available_clim].dropna(how='all')

    corr_values = np.zeros((len(available_phys), len(available_clim)))
    p_values    = np.zeros((len(available_phys), len(available_clim)))

    for i, pv in enumerate(available_phys):
        for j, cv in enumerate(available_clim):
            pair_df = sub_df[[pv, cv]].dropna()
            if len(pair_df) > 2:
                r, p = pearsonr(pair_df[pv], pair_df[cv])
            else:
                r, p = np.nan, np.nan
            corr_values[i,j] = r
            p_values[i,j]    = p

    corr_df = pd.DataFrame(corr_values, index=available_phys, columns=available_clim)
    pval_df = pd.DataFrame(p_values,  index=available_phys, columns=available_clim)

    corr_df.to_csv(os.path.join(output_dir, "ALL_physiol_clim_correlations.csv"))
    pval_df.to_csv(os.path.join(output_dir, "ALL_physiol_clim_pvalues.csv"))

    # Plot
    plt.figure(figsize=(12,6))
    sns.heatmap(corr_df, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title("PHYS-CLIM Correlations (Pearson R)\nAll Sessions Combined")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "ALL_physiol_clim_correlation_heatmap.png"), dpi=150)
    plt.close()

    print("Between-sessions correlation done. Saved to output folder.")

def main():
    # We'll do something similar to your existing approach:
    # We iterate over participants/sessions. For each session, we merge data, store in memory, do correlation, etc.

    # Example session map if needed
    sessions_map = {
       'Baixa': 4,
       'Belem': 1,
       'Parque': 6,
       'Gulbenkian': 3,
       'Lapa': 2,
       'Graca': 5
    }

    # We'll keep an all_data list to do the between-sessions approach
    all_data_frames = []
    search_dir = os.path.join(path.sourcedata, 'data')
    datadir = os.path.join(path.sourcedata, 'supp')
    for participant_folder in os.listdir(search_dir):
        if participant_folder.startswith("OE"):
            participant_path = os.path.join(search_dir, participant_folder)
            for session_folder in os.listdir(participant_path):
                session_path = os.path.join(participant_path, session_folder)
                if os.path.isdir(session_path):
                    session_name = extract_session_name(session_folder)
                    # Build the paths
                    subject_folder = f'sub-{participant_folder}'
                    output_dir = os.path.join(datadir, "correlation", subject_folder, f'ses-{session_name}')
                    empatica_data_path = os.path.join(datadir, 'stress_csv', subject_folder, f'ses-{session_name}', '_1hz', 'data_all_1Hz.csv')
                    climate_data_path  = os.path.join(datadir, 'geodata','log', subject_folder, f'ses-{session_name}', f'{subject_folder}_ses-{session_name}_geodata.xlsx')

                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir, exist_ok=True)

                    # 1) Merge
                    #   We'll do a simplified approach here or adapt from your existing "process_data" function
                    try:
                        # Load Empatica
                        empatica_data = pd.read_csv(empatica_data_path, parse_dates=['DateTime'])
                        empatica_data.dropna(subset=['DateTime'], inplace=True)
                        empatica_data.set_index('DateTime', inplace=True)

                        # Load climate
                        climate_data = pd.read_excel(climate_data_path, parse_dates=['time'])
                        climate_data.dropna(subset=['time'], inplace=True)
                        climate_data.set_index('time', inplace=True)

                        # Merge
                        combined_data = pd.merge(empatica_data, climate_data, left_index=True, right_index=True, how='inner')
                        # Possibly keep only numeric columns
                        # But let's keep all columns for referencing
                        combined_data.to_csv(os.path.join(output_dir, 'merged.csv'))
                        
                        # 2) Within-Session correlation
                        do_within_session_correlations(combined_data, session_name, output_dir)

                        # Keep for all-data approach
                        # We'll add a column 'Session' so we know from which session each row is
                        combined_data["Session"] = session_name
                        all_data_frames.append(combined_data)

                    except Exception as e:
                        print(f"Error merging or analyzing {participant_folder}-{session_name}: {e}")
                        continue

    # 3) Between-Session correlation
    if all_data_frames:
        big_df = pd.concat(all_data_frames, axis=0, ignore_index=False)
        # We'll do one big correlation
        out_all_dir = os.path.join(datadir, "correlation", "ALL_SESSIONS")
        os.makedirs(out_all_dir, exist_ok=True)
        do_between_sessions_correlations(big_df, out_all_dir)
    else:
        print("No data frames combined. Possibly no merges succeeded.")

    print("Done entire pipeline.")

if __name__ == "__main__":
    main()


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

PHYS_VARS = [
    "empatica_e4_gsr","empatica_e4_hr","empatica_e4_ibi","empatica_e4_temperature",
    # add more if needed
]

CLIM_VARS = [
    "tk_airquality_iaqindex_value","tk_airquality_temperature_value","tk_airquality_humidity_value",
    "tk_airquality_airpressure_value","tk_soundpressurelevel_spl_value","tk_humidity_humidity_value",
    "tk_analogin_voltage_value","tk_particulatematter_pm1_0_value","tk_particulatematter_pm2_5_value",
    "tk_particulatematter_pm10_0_value","tk_dual0_20ma_solarlight_value","tk_thermocouple_temperature_value",
    "tk_ptc_airtemp_value","atmos_northwind_value","atmos_eastwind_value","atmos_gustwind_value",
    "atmos_airtemperature_value","humidity","wind_speed","temp_atmos","temp_tk","temp_tk_ptc",
    "temp_radiant","utci","day_of_year","ghi","hPa","dew_point","solar_altitude","solar_azimuth",
    "mrt"
]

def accumulate_session_data(session_data_frames, session_name, combined_data):
    """
    Accumulate data for each session into a dictionary of lists. 
    We'll later concatenate them and compute one correlation matrix per session.
    """
    if session_name not in session_data_frames:
        session_data_frames[session_name] = []
    session_data_frames[session_name].append(combined_data)

def compute_session_correlation(df_session, session_name, outdir):
    """
    For the pooled data in a single session, compute correlation 
    among PHYS_VARS x CLIM_VARS and return the correlation DataFrame.
    """
    available_phys = [v for v in PHYS_VARS if v in df_session.columns]
    available_clim = [v for v in CLIM_VARS if v in df_session.columns]

    if not (available_phys and available_clim):
        print(f"No overlapping columns for session={session_name}. No correlation computed.")
        return None

    # Subset numeric columns
    sub_df = df_session[available_phys + available_clim].dropna(how='all')
    # Create a matrix (len(phys) x len(clim)) for correlation
    corr_values = np.zeros((len(available_phys), len(available_clim)))

    for i, pv in enumerate(available_phys):
        for j, cv in enumerate(available_clim):
            pair_df = sub_df[[pv, cv]].dropna()
            if len(pair_df) > 2:
                r, _ = pearsonr(pair_df[pv], pair_df[cv])
            else:
                r = np.nan
            corr_values[i,j] = r

    corr_df = pd.DataFrame(corr_values, index=available_phys, columns=available_clim)

    # Save CSV
    corr_df.to_csv(os.path.join(outdir, f"{session_name}_PHYS_CLIM_correlation.csv"))
    return corr_df

def main():
    # Example session names: 'Baixa', 'Belem', 'Parque', 'Gulbenkian', 'Lapa', 'Graca'
    # We'll store data per session for pooling:
    session_data_frames = {}  # e.g. { 'Lapa': [df1, df2,...], 'Parque': [...], ... }

    # We'll also keep a global list for "all sessions" if needed
    all_data_frames = []

    # Example search dir
    search_dir = os.path.join(path.sourcedata, 'data')
    datadir = os.path.join(path.sourcedata, 'supp')
    for participant_folder in os.listdir(search_dir):
        if participant_folder.startswith("OE"):
            participant_path = os.path.join(search_dir, participant_folder)
            for session_folder in os.listdir(participant_path):
                session_path = os.path.join(participant_path, session_folder)
                if os.path.isdir(session_path):
                    session_name = extract_session_name(session_folder)
                    if session_name is None:
                        continue

                    # Build hypothetical paths
                    subject_folder = f'sub-{participant_folder}'
                    empatica_data_path = os.path.join(datadir, 'stress_csv', subject_folder, f'ses-{session_name}', '_1hz', 'data_all_1Hz.csv')
                    climate_data_path  = os.path.join(datadir, 'geodata','log', subject_folder, f'ses-{session_name}', f'{subject_folder}_ses-{session_name}_geodata.xlsx')

                    if not os.path.exists(empatica_data_path) or not os.path.exists(climate_data_path):
                        print(f"Missing data for {participant_folder}-{session_name}")
                        continue

                    try:
                        # Load Empatica
                        empatica_data = pd.read_csv(empatica_data_path, parse_dates=['DateTime'])
                        empatica_data.dropna(subset=['DateTime'], inplace=True)
                        empatica_data.set_index('DateTime', inplace=True)

                        # Load climate
                        climate_data = pd.read_excel(climate_data_path, parse_dates=['time'])
                        climate_data.dropna(subset=['time'], inplace=True)
                        climate_data.set_index('time', inplace=True)

                        # Merge
                        combined_data = pd.merge(empatica_data, climate_data,
                                                 left_index=True, right_index=True, how='inner')
                        # Add a 'Session' column
                        combined_data["Session"] = session_name

                        # Accumulate for session-level
                        accumulate_session_data(session_data_frames, session_name, combined_data)
                        # Accumulate for all-sessions
                        all_data_frames.append(combined_data)

                    except Exception as e:
                        print(f"Error merging {participant_folder}-{session_name}: {e}")
                        continue

    # ---------------------------------------------------------------------
    # Now we create an output directory to store session correlations 
    # (and the final multi-plot)
    # ---------------------------------------------------------------------
    outdir = os.path.join(datadir, "all_sessions_correlations")
    os.makedirs(outdir, exist_ok=True)

    # For each session, combine data from all participants
    # then compute correlation => store in a dictionary
    session_corrs = {}
    for ses, list_of_dfs in session_data_frames.items():
        if not list_of_dfs:
            continue
        big_ses_df = pd.concat(list_of_dfs, ignore_index=False)
        corr_df = compute_session_correlation(big_ses_df, ses, outdir)
        if corr_df is not None:
            session_corrs[ses] = corr_df

    # Now we create a single figure with one subplot for each session
    # Suppose we have up to 6 sessions, we can arrange them in (2,3) grid
    sessions_sorted = sorted(session_corrs.keys())  # sort alphabetically or otherwise
    n_ses = len(sessions_sorted)

    fig_cols = 3 if n_ses>3 else n_ses
    fig_rows = int(np.ceil(n_ses/fig_cols))

    fig, axes = plt.subplots(fig_rows, fig_cols, figsize=(6*fig_cols,5*fig_rows))
    axes = np.atleast_2d(axes)  # ensure 2D for indexing

    for i, ses in enumerate(sessions_sorted):
        row = i // fig_cols
        col = i % fig_cols
        ax = axes[row, col]

        corr_df = session_corrs[ses]
        sns.heatmap(corr_df, annot=False, cmap="coolwarm", vmin=-1, vmax=1, ax=ax)
        ax.set_title(f"Session: {ses}", fontsize=12)

    # If there's any leftover subplot, hide it
    if n_ses < fig_rows*fig_cols:
        for empty_i in range(n_ses, fig_rows*fig_cols):
            er = empty_i//fig_cols
            ec = empty_i%fig_cols
            axes[er,ec].set_visible(False)

    plt.suptitle("PHYS-CLIM Correlation Matrices by Session", fontsize=14)
    plt.tight_layout(rect=[0,0,1,0.96])
    plt.savefig(os.path.join(outdir, "multi_session_correlations.png"), dpi=150)
    plt.close()

    # Optionally do the "between-sessions" correlation 
    # by pooling all data from all_data_frames
    if all_data_frames:
        big_df = pd.concat(all_data_frames, ignore_index=False)
        # We'll do the same approach as 'compute_session_correlation' 
        # but call it "ALL"
        all_corr_df = compute_session_correlation(big_df, "ALL", outdir)
        if all_corr_df is not None:
            plt.figure(figsize=(12,6))
            sns.heatmap(all_corr_df, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
            plt.title("PHYS-CLIM Correlation (All Sessions Pooled)")
            plt.tight_layout()
            plt.savefig(os.path.join(outdir, "ALL_sessions_correlation_heatmap.png"), dpi=150)
            plt.close()

    print("Done. Multi-subplot correlation figure created for each session.")


if __name__ == "__main__":
    main()
